In [ ]:
%matplotlib inline
from run_dnerf import config_parser, create_nerf
import torch
from load_blender import pose_spherical
from run_dnerf import render_path
from run_dnerf_helpers import to8b
import numpy as np
import matplotlib.pyplot as plt
import mcubes
import trimesh

In [ ]:
# set cuda
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# get config file
config_file = "configs/sim_data.txt"
parser = config_parser()
args = parser.parse_args(f'--config {config_file}')

# set render params
hwf = [400, 400, 555.555]
_, render_kwargs_test, _, _, _ = create_nerf(args)
render_kwargs_test.update({'near' : 2., 'far' : 6.})

## Generate Debug Frame

In [ ]:
def generate_img(time):
    render_poses = torch.unsqueeze(pose_spherical(180, -20, 4.0), 0).to(device)
    #render_poses = torch.unsqueeze(pose_spherical(135, -20, 4.0), 0).to(device)
    render_times = torch.Tensor([time]).to(device)

    with torch.no_grad():
            rgbs, _ = render_path(render_poses, render_times, hwf, args.chunk, render_kwargs_test, render_factor=args.render_factor)
    rgbs = to8b(rgbs)
    return rgbs[0]

In [ ]:
# genearte
time = 0.0  # time in [0,1]
img_0 = generate_img(time)

# display
plt.figure(2, figsize=(20,6))
plt.imshow(img_0)
plt.show()

In [ ]:
# genearte
time = .5  # time in [0,1]
img = generate_img(time)

# display
plt.figure(2, figsize=(20,6))
plt.imshow(img)
plt.show()

In [ ]:
import cv2
def get_diff_image(a,b):
    img_a = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
    img_b = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
    diff = np.abs(img_a - img_b)
    return diff

# display
plt.figure(2, figsize=(20,6))
plt.imshow(get_diff_image(img_0, img), cmap = 'hot' , interpolation = 'nearest' )
plt.show()

In [ ]:
save_dir = '/data3/ankit/results_simdata'

In [ ]:
#Create video
plt.close()
import os
import imageio

prev = img_0.copy()

if not os.path.exists(save_dir):
    os.mkdir(save_dir)
img_file_names = []
diff_file_names = []
for i in np.arange(0.,1.0,0.1):
    img = generate_img(i)
    f_path = os.path.join(save_dir, 'image_{}.png'.format(i))
    plt.figure(2, figsize=(20,6))
    plt.imshow(img)
    plt.savefig(f_path)
    plt.close()
    img_file_names.append(f_path)
    
    f_path = os.path.join(save_dir, 'diff_image_{}.png'.format(i))
    plt.figure(2, figsize=(20,6))
    plt.imshow(get_diff_image(img, prev), cmap = 'hot' , interpolation = 'nearest' )
    plt.savefig(f_path)
    plt.close()
    diff_file_names.append(f_path)
    prev = img.copy()

In [ ]:
import imageio
def create_gif(gif_name, list_files):
    with imageio.get_writer(os.path.join(save_dir,gif_name), mode='I') as writer:
        for file_ in list_files:
            image = imageio.imread(file_)
            writer.append_data(image)

print(len(img_file_names))
print(len(diff_file_names))
create_gif('hw_images.gif', img_file_names)
create_gif('hw_diff.gif', diff_file_names)

In [ ]:
input_dir = '/data3/ankit/Experiments/NERF/D-NeRF/sim_data'
file_lists = []
for i in range(51):
    file_lists.append(os.path.join(input_dir, 'image_{}.png'.format(i)))
create_gif('input_images.gif', file_lists)

## Reconstruct

In [ ]:
N = 256
min_rec, max_rec = -1.2, 1.2
t = np.linspace(min_rec, max_rec, N+1)

query_pts = np.stack(np.meshgrid(t, t, t), -1).astype(np.float32)
sh = query_pts.shape
pts = torch.Tensor(query_pts.reshape([-1,3]))


def batchify(fn, chunk):
    if chunk is None:
        return fn
    def ret(inputs):
        return torch.concat([fn(inputs[i:i+chunk]) for i in range(0, inputs.shape[0], chunk)], 0)
    return ret

def reconstruct(points, time):
    net_fn = render_kwargs_test['network_query_fn']
    fn = lambda i0, i1 : net_fn(points[i0:i1,None,:], viewdirs=torch.zeros_like(points[i0:i1]), ts=torch.ones_like(points[i0:i1])[:,0:1]*time, network_fn=render_kwargs_test['network_fn'])
    chunk = 1024*64
    raw = np.concatenate([fn(i, i+chunk)[0].cpu().numpy() for i in range(0, points.shape[0], chunk)], 0)
    raw = np.reshape(raw, list(sh[:-1]) + [-1])
    sigma = np.maximum(raw[...,-1], 0.)
    return sigma
    
with torch.no_grad():
    sigma = reconstruct(pts, time)

In [ ]:
# reconstruct
threshold = 40  # this is just a randomly found threshold
vertices, triangles = mcubes.marching_cubes(sigma, threshold)

# display
mesh = trimesh.Trimesh(vertices / N * (max_rec - min_rec) + min_rec , triangles)
mesh = trimesh.smoothing.filter_laplacian(mesh, iterations=3)
mesh.show()